# Goals: 
 - Take in filtered JSON files and output them to df and do analysis on

In [ ]:
from tqdm import tqdm
import json
import numpy as np
import pandas as pd

In [ ]:
with open('Ngrams/1-00015-of-00024-lemmatized.json', 'r') as f:
    ngrams = json.load(f)
    f.close()

In [ ]:
words = list(ngrams.keys())
years = [str(i) for i in range(1800,2020)]

In [ ]:
def open_json(directory,file_path):
    with open(directory+file_path,'r') as f:
        ngrams = json.load(f)
        f.close()
    return ngrams

In [ ]:
def normalize(ngrams):
    words = list(ngrams.keys())
    years = [str(i) for i in range(1800,2020)]
    unigram_dict = dict()
    i=0
    for word in words:
        match_count_by_year = []
        for year in years:
            if year in ngrams[word].keys():
                match_count_by_year.append(ngrams[word][year])
            else:
                #Zeroes are necessary for smoothing
                match_count_by_year.append(0)
        unigram_dict[word] = match_count_by_year
        del match_count_by_year
        i+=1
        if i%50000==0:
            print(i)
    
    return unigram_dict, words, years

In [ ]:
def df_and_smooth(unigram_dict, years):
    return pd.DataFrame.from_dict(unigram_dict #take in the dictionary
                ).rolling(5,center=True #create frames of size 5 (smoothing value), and replace value in middle
                ).mean( #average accross those frames
                ).rename({i:years[i] for i in range(len(years))}, axis = 'index' #rename the indices to years
                ).dropna( #drop the edge years that were not averaged with 4 other rows
                ).replace(to_replace=0, value=np.NaN) #replace all zeroes with NaN for the next iteration step

Normalize list so that there are zeros for every year, which is necessary so that the lists are the same size and for smoothing

In [ ]:
%%time
unigram_dict = {}
for word in tqdm(words):
    match_count_by_year = []
    for year in years:
        if year in ngrams[word].keys():
            match_count_by_year.append(ngrams[word][year])
        else:
            #Zeroes are necessary for smoothing
            match_count_by_year.append(0)
    unigram_dict[word] = match_count_by_year
    del match_count_by_year
del ngrams

In [ ]:
%%time
df = pd.DataFrame.from_dict(unigram_dict #take in the dictionary
                ).rolling(5,center=True #create frames of size 5 (smoothing value), and replace value in middle
                ).mean( #average accross those frames
                ).rename({i:years[i] for i in range(len(years))}, axis = 'index' #rename the indices to years
                ).dropna() #drop the edge years that were not averaged with 4 other rows
                #).replace(to_replace=0, value=np.NaN) #replace all zeroes with NaN for the next iteration step
del years
del unigram_dict

In [ ]:
df = pd.DataFrame.from_dict(unigram_dict)
del unigram_dict

In [ ]:
words[:10]

In [ ]:
%%time
years_map = {i:int(year) for i, year in enumerate(df.index)}
ngrams_processed = df.to_dict(orient = 'list')
del df

In [ ]:
ngrams_processed

In [ ]:
%%time
birth, death = {},{}
for unigram in tqdm(ngrams_processed):
    l = ngrams_processed[unigram]
    birth_years, death_years = [],[]
    for i in range(len(l)-1):
        #Birth
        if l[i]==0 and l[i+1]!=0:
            birth_years.append(years_map[i+1])
        #Death
        if l[i]!=0 and l[i+1]==0:
            death_years.append(years_map[i])
    
        #Disregarding death in the final year
            
    if len(birth_years)>0:
        birth[unigram] = birth_years

    if len(death_years)>0:
        death[unigram] = death_years

print(len(birth),len(death))

In [ ]:
birth

In [ ]:
death

In [ ]:
l = ngrams_processed['trichiulus']
birth, death = {},{}
birth_years, death_years = [],[]
for i in range(len(l)-1):
    #Birth
    if l[i]==0 and l[i+1]!=0:
        birth_years.append(years_map[i+1])
    #Death
    if l[i]!=0 and l[i+1]==0:
        death_years.append(years_map[i])

    #Disregarding death in the final year

if len(birth_years)>0:
    birth[word] = birth_years

if len(death_years)>0:
    death[word] = death_years

In [ ]:
birth

Put everything together

In [ ]:
def save_json(dictionary,directory,file_path):
    output = file_path+'.json'
    if len(dictionary)>0:
        with open(directory+output, 'w') as f_out:
            json.dump(ngram_dict, f_out)
        print('SAVED: ',output,len(ngram_dict))
    else:
        print('unigram dict empty',output)

In [ ]:
#Put the words that birth and die into a dictionariess with the years of birth/death in separated lists
birth,death = {},{}
i=0
for word in words:
    #If there were zeroes in the series proceed, else skip 
    if df[word].hasnans:
        print('good word',i)
        birth_years, death_years = [],[]
        
        for idx, val in df[word].items():
            print(idx,val,i)
            if idx == df[word].index[len(df[word])-1]:
                #Death
                if not pd.isna(df[word].at[str(int(idx)-1)]) and pd.isna(df[word].at[idx]):
                    death_years.append(int(idx)-1)

            else:
                #Birth
                if pd.isna(df[word].at[idx]) and not pd.isna(df[word].at[str(int(idx)+1)]):
                    print('birthed')
                    birth_years.append(int(idx)+1)
                
                #Death
                if not pd.isna(df[word].at[str(int(idx))]) and pd.isna(df[word].at[str(int(idx)+1)]):
                    print('dead')
                    death_years.append(int(idx))
                
        birth[word] = birth_years
        death[word] = death_years
    i+=1
    if i%50000==0:
        print(birth)
        print(death)
        print(i)

In [ ]:
birth

In [ ]:
death

### Unused Code

Calculate rolling average

In [ ]:
df_ave=df.rolling(5,center=True).mean().rename({i:years[i] for i in range(len(years))}, axis = 'index').dropna().copy()
del df
del years

In [ ]:
df_ave.tail(16)

In [ ]:
    for idx, val in df_ave['trichiulus'].items():
    
    if idx == df_ave['trichiulus'].index[len(df_ave['trichiulus'])-1]:
        print(idx,val)
        pass
    else:
        print(idx,val)
        print('')
        print('next',df_ave['trichiulus'].at[str(int(idx)+1)])
        #if val: 

In [ ]:
type(df_ave['trichiulus'])
#Series.hasnans

In [ ]:
df['trichiulus']

In [ ]:
df[word].at['2015']

In [ ]:
pd.isna(df[word].at['2015'])

In [ ]:
type(df[word].at['2015'])

In [ ]:
type(np.nan)

In [ ]:
df['trichiulus'].index[len(df['trichiulus'])-1]

[index from integer location](https://stackoverflow.com/questions/17244049/finding-label-location-in-a-dataframe-index)

In [ ]:
for idx, val in df_ave['trichiulus'].items():
    
    if idx == df_ave['trichiulus'].index[len(df_ave['trichiulus'])-1]:
        print(idx,val)
        pass
    else:
        print(idx,val)
        print('')
        print('next',df_ave['trichiulus'].at[str(int(idx)+1)],type(df_ave['trichiulus'].at[str(int(idx)+1)]))
        #if val: 

In [ ]:
df_ave.replace(to_replace=0, value=np.NaN)

In [ ]:
#working rename command for end
df.rename({i:years[i] for i in range(len(years))}, axis = 'index')

In [ ]:
df_ave=df.rolling(2,center=True,axis=1,closed='both').mean().copy()
del df

In [ ]:
df_ave[:10]

In [ ]:
for row in df.iterrows():
    print(row)
    print(type(row))
    break

In [ ]:
directory = './Ngrams/'
files = os.listdir(directory)
for file_path in files:
    if '.gz' in file_path:
        preprocess_ngrams(directory,file_path)